In [1]:
# Import packaages for financial risk modeling
from pylab import *
import yfinance
from datetime import datetime
import datetime as dt
import pandas_datareader as web
from datetime import datetime
from pandas_datareader.data import DataReader
from datetime import date
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm
import math

In [2]:
# Display options
pd.options.display.width = 0

In [3]:
# Set start date
end = datetime.now()
start = date(2001, 1, 1)

In [4]:
# Set the ticker
ticker = 'AAPL', 'AOS', 'SPY', '^FVX'

In [5]:
sec = yfinance.download(ticker, start=start, end=end)['Adj Close']
sec = pd.DataFrame(sec)

[*********************100%***********************]  4 of 4 completed


In [6]:
# Display result
print(sec.head(3))

                AAPL       AOS        SPY   ^FVX
Date                                            
2001-01-02  0.226078  1.907969  85.301773  4.752
2001-01-03  0.248876  1.978634  89.399254  4.923
2001-01-04  0.259326  1.985700  88.436981  4.808


In [7]:
# Join stock dataframes together
full_df = pd.concat([sec], axis=1).dropna()

In [8]:
# Resample the full dataframe to monthly timeframe
monthly_df = full_df.resample('BMS').first()
print(monthly_df)

                  AAPL        AOS         SPY   ^FVX
Date                                                
2001-01-01    0.226078   1.907969   85.301773  4.752
2001-02-01    0.321069   1.960338   91.339539  4.723
2001-03-01    0.284972   2.113062   82.512207  4.628
2001-04-02    0.328137   2.160931   75.828751  4.595
2001-05-01    0.394098   2.203320   84.361137  4.850
...                ...        ...         ...    ...
2022-10-03  141.997284  50.087154  364.934204  3.883
2022-11-01  150.171219  54.328625  382.762329  4.258
2022-12-01  148.083893  60.179401  405.517853  3.680
2023-01-02  124.879326  58.627155  380.820007  3.945
2023-02-01  145.208282  69.959999  410.799988  3.495

[266 rows x 4 columns]


In [9]:
returns = monthly_df[[key for key in dict(monthly_df.dtypes) if dict(monthly_df.dtypes)[key] in ['float64', 'int64']]].pct_change()
returns = returns[1:]
print(returns.head())
returns['Intercept'] = 1
print(returns.head(3))

                AAPL       AOS       SPY      ^FVX
Date                                              
2001-02-01  0.420168  0.027448  0.070781 -0.006103
2001-03-01 -0.112427  0.077907 -0.096643 -0.020114
2001-04-02  0.151469  0.022654 -0.081000 -0.007131
2001-05-01  0.201019  0.019616  0.112522  0.055495
2001-06-01 -0.194369 -0.109375 -0.002518  0.012784
                AAPL       AOS       SPY      ^FVX  Intercept
Date                                                         
2001-02-01  0.420168  0.027448  0.070781 -0.006103          1
2001-03-01 -0.112427  0.077907 -0.096643 -0.020114          1
2001-04-02  0.151469  0.022654 -0.081000 -0.007131          1


In [10]:
# Estimating historical risk
stockNames = ['AAPL', 'AOS']
factorNames = ['SPY', '^FVX', 'Intercept']

In [11]:
stockReturns = returns[stockNames]
factorReturns = returns[factorNames]
weights = np.array([1.0/len(stockNames)] * len(stockNames))

In [12]:
historicalTotalRisk = np.dot(np.dot(weights, stockReturns.cov()), weights.T)
print(historicalTotalRisk)

0.005815039785997518


In [13]:
# Building Factor Models
xData = factorReturns
print(xData.head())

                 SPY      ^FVX  Intercept
Date                                     
2001-02-01  0.070781 -0.006103          1
2001-03-01 -0.096643 -0.020114          1
2001-04-02 -0.081000 -0.007131          1
2001-05-01  0.112522  0.055495          1
2001-06-01 -0.002518  0.012784          1


In [14]:
modelCoeffs = []
for oneStockName in stockNames:
  yData = stockReturns[oneStockName]
  model = sm.OLS(yData, xData)
  result = model .fit()
  modelCoeffRow = list(result.params)
  modelCoeffRow.append(np.std(result.resid, ddof=1))
  modelCoeffs.append(modelCoeffRow)
  print(result.summary())
modelCoeffs = pd.DataFrame(modelCoeffs)
modelCoeffs.columns = ['B_SPY', 'B_FVX', 'Alpha', 'ResidVol']
modelCoeffs['Names'] = stockNames
print(modelCoeffs)

                            OLS Regression Results                            
Dep. Variable:                   AAPL   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     61.11
Date:                Sun, 12 Feb 2023   Prob (F-statistic):           1.65e-22
Time:                        14:42:21   Log-Likelihood:                 274.53
No. Observations:                 265   AIC:                            -543.1
Df Residuals:                     262   BIC:                            -532.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SPY            1.2342      0.114     10.858      0.0

In [15]:
# Calculate Idiosyncratic and Systemic Risk
factorCov = factorReturns[['^FVX', 'SPY']].cov()
reconstructedCov = np.dot(np.dot(modelCoeffs[['B_FVX', 'B_SPY']], factorCov), modelCoeffs[['B_FVX', 'B_SPY']].T)
systemicRisk = np.dot(np.dot(weights, reconstructedCov), weights.T)
print('Systemic Risk')
print(systemicRisk)
idiosyncraticRisk = sum(modelCoeffs['ResidVol'] * modelCoeffs['ResidVol'] * weights * weights)
print('Idiosyncratic Risk')
print(idiosyncraticRisk)
factorModelTotalRisk = systemicRisk + idiosyncraticRisk
print('Factor Model Total Risk')
print(factorModelTotalRisk)

Systemic Risk
0.002697320792201207
Idiosyncratic Risk
0.003251961848331433
Factor Model Total Risk
0.00594928264053264


In [16]:
# Scenario based stress testing
fvxScenarios = np.arange(min(returns['^FVX']), max(returns['^FVX']), 0.05)
spScenarios = np.arange(min(returns['SPY']), max(returns['SPY']), 0.02)

scenarios = []
for oneFVXValue in fvxScenarios:
  for oneSPValue in spScenarios:
     oneScenario = [oneFVXValue, oneSPValue]
     for oneStockName in stockNames:
        alpha = float(modelCoeffs[modelCoeffs['Names'] == oneStockName]['Alpha'])
        beta_sp = float(modelCoeffs[modelCoeffs['Names'] == oneStockName]['B_SPY'])
        beta_fvx = float(modelCoeffs[modelCoeffs['Names'] == oneStockName]['B_FVX'])
        oneStockPredictedReturn = alpha + beta_sp * oneSPValue + beta_fvx * oneFVXValue
        oneScenario.append(oneStockPredictedReturn)
     scenarios.append(oneScenario)
scenarios = pd.DataFrame(scenarios)
scenarios.columns = ['^FVX', 'SPY', 'AAPL', 'AOS']
scenariosCov = scenarios[stockNames].cov()
scenarioTotalRisk = np.dot(np.dot(weights, scenariosCov), weights.T)

In [17]:
# Quantify worst case scenario with VaR
confLevel = 0.95
principal = 1
numMonths = 1

In [18]:
def calculateVaR(risk, confLevel, principal=1, numMonths=1):
  vol = math.sqrt(risk)
  return abs(principal*norm.ppf(1-confLevel, 0, 1)*vol*math.sqrt(numMonths))

print('Scenario Total Risk VaR')
print(calculateVaR(scenarioTotalRisk, 0.99))
print('Historical Total Risk VaR')
print(calculateVaR(historicalTotalRisk, 0.99))
print('Factor Model Total Risk VaR')
print(calculateVaR(factorModelTotalRisk, 0.99))

Scenario Total Risk VaR
0.2576345855446903
Historical Total Risk VaR
0.17739893301313678
Factor Model Total Risk VaR
0.17943491742430415
